In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,029 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,593 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-08-31 00:00:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [6]:
high_votes_df = vine_df.filter("total_votes>=20")
high_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R17V95WYC9ZXDD|          1|            0|         20|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|     

In [10]:
helpful_df = high_votes_df.filter(high_votes_df.helpful_votes / high_votes_df.total_votes >= .5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [11]:
paid_df = helpful_df.filter(helpful_df.verified_purchase == 'Y')
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| RIRX9H2QJFD1P|          5|           49|         53|   N|                Y|
|R39R2LEKGIEUFS|          5|           69|         77|   N|                Y|
|R1IN7ZA9ZCZEVM|          5|           17|         25|   N|                Y|
|R253TQ098JL914|          2|           22|         23|   N|                Y|
|R3OPTDO4EDS6LN|          5|           30|         30|   N|                Y|
|R3NJE21NUCII5L|          5|          135|        144|   N|     

In [12]:
unpaid_df = helpful_df.filter(helpful_df.verified_purchase == 'N')
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|                N|
|R2NCX85FTSTWHC|          5|           65|         68|   N|                N|
|R1FGHHDQ4P21C0|          1|           30|         33|   N|                N|
|R3FQ1CSDIOWWRY|          1|           71|         80|   N|                N|
|R29H3IX0EL2VZR|          4|           20|         23|   N|     

In [22]:
# Stats for paid reviews
from pyspark.sql.functions import sum
paid_df.select(sum(paid_df.total_votes).alias("Total Reviews")).show()
paid_df.filter("star_rating=5").select(sum(paid_df.total_votes).alias("Total 5 Star Reviews")).show()
print("5 star reviews make up " + str(round(21183/49035*100, 2)) + "% of the total paid reviews")


+-------------+
|Total Reviews|
+-------------+
|        49035|
+-------------+

+--------------------+
|Total 5 Star Reviews|
+--------------------+
|               21183|
+--------------------+

5 star reviews make up 43.2% of the total paid reviews


In [24]:
# Stats for unpaid reviews
unpaid_df.select(sum(unpaid_df.total_votes).alias("Total Reviews")).show()
unpaid_df.filter("star_rating=5").select(sum(unpaid_df.total_votes).alias("Total 5 Star Reviews")).show()
print("5 star reviews make up " + str(round(18129/54237*100, 2)) + "% of the total unpaid reviews")

+-------------+
|Total Reviews|
+-------------+
|        54237|
+-------------+

+--------------------+
|Total 5 Star Reviews|
+--------------------+
|               18129|
+--------------------+

5 star reviews make up 33.43% of the total unpaid reviews
